Binyu Yang

Objective Two: Determine the summary statistics of each variable in your dataset that you plan to use for analysis.

In [8]:
!pip install pyreadr


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pyreadr
import pandas as pd
import numpy as np

In [2]:
# Import datasets

# FIRE ON datasets
on_1029 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireON_join_20181029.rds")[None]
on_1110 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireON_join_20181110.rds")[None]
on_1204 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireON_join_20181204.rds")[None]

# FIRE OFF datasets
off_1029 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireOFF_join_20181029.rds")[None]
off_1110 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireOFF_join_20181110.rds")[None]
off_1204 = pyreadr.read_r(r"C:\Users\byang77\OneDrive - Emory\Desktop\Python_Final\CMAQ_fireOFF_join_20181204.rds")[None]


PyreadrError: File b'C:\\Users\\byang77\\OneDrive - Emory\\Desktop\\Python_Final\\CMAQ_fireON_join_20181029.rds' does not exist!

In [ ]:
# Create summary statistics

def summarize(df):
    return pd.DataFrame({
        "mean": df.mean(),
        "median": df.median(),
        "std": df.std(),
        "min": df.min(),
        "max": df.max(),
        "IQR": df.quantile(0.75) - df.quantile(0.25),
        "missing": df.isna().sum()
    })

datasets = {
    "on_1029": on_1029,
    "on_1110": on_1110,
    "on_1204": on_1204,
    "off_1029": off_1029,
    "off_1110": off_1110,
    "off_1204": off_1204
}

summary_tables = {}

for name, df in datasets.items():
    df_numeric = df[["benzene", "form", "lon", "lat"]]
    summary_tables[name] = summarize(df_numeric)

summary_all = pd.concat(summary_tables, axis=0)
summary_all


Updated 3 aims:

RQ1. How much do benzene and formaldehyde concentrations increase when wildfire emissions are included (Fire-ON) compared to excluded (Fire-OFF) for each of the three study days?
RQ2. Which geographic regions show the largest wildfire-driven increases in benzene and formaldehyde concentrations (ON–OFF), and do these spatial patterns differ between pollutants?
RQ3. Do wildfire-attributable pollution levels (Fire-ON minus Fire-OFF) change across the three time points representing before, during, and after peak wildfire activity?

Objective Three: Create a Statistical Analysis Plan (SAP) based on the outcomes from objective two, the questions your posed in final part one and your statistical knowledge. Provide as much detail as possible about the statistical and regression techniques you will need to use to answer the questions of interest.

The statistical analysis for this project is designed to evaluate wildfire-attributable changes in air pollutant concentrations by comparing output from CMAQ model simulations conducted with and without wildfire emissions. The analyses address three aims and rely on paired pollutant estimates across identical spatial grid cells, allowing within-cell comparisons across both fire scenarios and time points. All analyses will be conducted using Python, and modeling choices and statistical procedures were selected based on the structure of the dataset, the deterministic nature of CMAQ output, and the need to characterize spatial and temporal differences descriptively rather than infer causality.

To address the first aim—quantifying whether pollutant concentrations differ between the Fire-ON and Fire-OFF simulations for each day—paired statistical comparisons will be used. Because each grid cell has both a Fire-ON and Fire-OFF estimate for benzene and formaldehyde, the two scenarios form a natural pair, enabling direct cell-level comparison without the need for additional adjustment. For each pollutant and each of the three dates (October 29, November 10, and December 4), a difference variable will be calculated by subtracting the Fire-OFF concentration from the corresponding Fire-ON concentration. This difference represents the wildfire-attributable increment in pollutant concentrations. The distributions of these difference values will first be summarized using standard descriptive measures, including the mean, median, standard deviation, minimum, maximum, and interquartile range. These descriptive summaries are necessary to evaluate the magnitude of changes, detect outliers, and understand whether wildfire emissions increased overall pollutant levels.

In addition to descriptive summaries, a paired t-test will be applied to formally assess whether the Fire-ON and Fire-OFF concentrations differ at the population level for each pollutant and date. Although CMAQ outputs are deterministic and thus not strictly subject to sampling variability, the paired t-test provides a standardized framework for quantifying the magnitude and direction of differences across thousands of grid cells. This test evaluates whether the mean of the paired differences differs from zero. The test will be implemented using SciPy’s ttest_rel function, which accounts for the paired structure and provides the t-statistic and p-value. The assumptions of the paired t-test—namely, approximate normality of differences—will be evaluated visually using histogram and kernel density plots of the difference distributions. If difference distributions appear strongly skewed, a nonparametric paired test such as the Wilcoxon signed-rank test may be considered, though such deviations are unlikely to affect interpretability given the large sample size.

The second aim seeks to identify spatial patterns of wildfire-attributable pollution across California. To evaluate spatial variation, the Fire-ON minus Fire-OFF difference variables will be linked to the latitude and longitude coordinates for each grid cell. Because these coordinates already define a consistent spatial grid, no spatial transformation is required. Spatial variability will be assessed descriptively by examining how difference values change across the spatial domain, with particular attention to regions known to be closer to major 2018 fire events. Planned visualizations for subsequent objectives include spatial scatter maps or heatmaps constructed using matplotlib or geopandas, in which difference magnitudes are represented with color gradients. These maps will allow identification of pollution “hotspots” attributable to wildfire emissions. Although the core purpose of Aim 2 is descriptive, a simple spatial gradient model may also be explored by regressing difference values on latitude and longitude using an ordinary least squares model from the statsmodels package. This model will provide insight into whether wildfire impacts vary systematically along north–south or east–west axes. Spatial autocorrelation tests (e.g., Moran’s I) could be considered for completeness, though they are not required for the aims of this project.

The third aim evaluates temporal changes in wildfire-attributable pollution across the three representative dates. Because each date corresponds to a separate CMAQ simulation rather than repeated measurements over time, the data do not constitute a longitudinal dataset and will not be analyzed using repeated-measures or mixed-effects models. Instead, temporal comparisons will be conducted by summarizing and comparing the distributions of difference variables across the three days. For each pollutant, the mean and median difference values will be compared qualitatively across October 29 (pre-fire), November 10 (peak fire activity), and December 4 (post-fire). Boxplots and kernel density estimates, to be generated using seaborn, will be planned for later visualization and will allow inspection of shifts in the central tendency and spread of wildfire-attributable pollution over time. If clear temporal gradients are observed, additional descriptive comparisons such as percent change from pre-fire to peak-fire conditions may be computed.

All analyses will be conducted in Python. Data will be imported using pyreadr, managed using pandas, and analyzed numerically using numpy. Statistical tests will be performed using SciPy, and optional regression models will employ the statsmodels library. Visualizations, including spatial scatter plots, KDE plots, and boxplots, will be generated in later objectives using matplotlib, seaborn, and optionally geopandas and contextily for geospatial enhancement. These procedures collectively ensure that all three analytic aims are addressed rigorously, transparently, and in alignment with the deterministic nature of the CMAQ modeling data.

Object 3： Analysis

Aim 1: Comparison of Fire-ON and Fire-OFF concentrations

To evaluate the wildfire-attributable increase in pollutant concentrations, we compared benzene and formaldehyde levels between the Fire-ON and Fire-OFF CMAQ simulations for each of the three study dates. Because the model provides estimates for identical grid cells across both scenarios, the values form natural pairs, allowing cell-level comparison.

For each pollutant, we calculated the difference (Fire-ON minus Fire-OFF). Summary statistics of these differences were generated to characterize the magnitude of wildfire impact. We additionally conducted paired t-tests to formally assess whether Fire-ON concentrations differed from Fire-OFF concentrations for each date. Although CMAQ outputs are deterministic, paired t-tests provide a standardized quantitative comparison across grid cells.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

datasets_on = {
    "1029": on_1029,
    "1110": on_1110,
    "1204": on_1204
}

datasets_off = {
    "1029": off_1029,
    "1110": off_1110,
    "1204": off_1204
}

diff_results = {}

In [ ]:
for date in ["1029", "1110", "1204"]:
    df_on = datasets_on[date]
    df_off = datasets_off[date]

    df_diff = pd.DataFrame({
        "benzene_diff": df_on["benzene"] - df_off["benzene"],
        "form_diff": df_on["form"] - df_off["form"]
    })

    diff_results[date] = df_diff